In [1]:
import os

In [2]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\Youtube Project\\research'

In [3]:
os.chdir('..')

In [4]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\Youtube Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# Dataclass is just a decorator that automatically adds special methods to classes, 
# like __init__() and __repr__(), based on class attributes. 

# It simplifies the creation of classes that primarily store data.
@dataclass(frozen=True)
class dataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])

    def get_data_ingestion_config(self) -> dataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = dataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [7]:
import zipfile
import os
import gdown
from src.cnnClassifier import logger

class DataIngestion:
    def __init__(self, config: dataIngestionConfig):
        self.config = config

    def download_file(self) -> Path:
        if os.path.exists(self.config.local_data_file):
            logger.info(f"File already exists at {self.config.local_data_file}. Skipping download.")
        else:
            logger.info(f"Downloading file from {self.config.source_URL} to {self.config.local_data_file}")
            gdown.download(url=self.config.source_URL, output=self.config.local_data_file, quiet=False)
            logger.info(f"File downloaded successfully to {self.config.local_data_file}")
        return self.config.local_data_file

    def extract_zip_file(self, zip_file_path: Path):
        logger.info(f"Extracting zip file {zip_file_path} to directory {self.config.unzip_dir}")
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info(f"Extraction completed successfully to directory {self.config.unzip_dir}")

    def initiate_data_ingestion(self):
        zip_file_path = self.download_file()
        self.extract_zip_file(zip_file_path)
        return self.config.unzip_dir